In [1]:
# Importing the utilites
import nltk
from nltk.stem import WordNetLemmatizer
import spacy
from scipy.sparse import vstack
import json
import pickle
import yaml
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import tensorflow as tf 
import re
import random

In [2]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lakshya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lakshya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Lakshya\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

In [5]:
data_file = open("G:\Code_clause\Chatbot\models\Data\intent.json",'r',encoding = 'UTF-8').read()
intents = json.loads(data_file)

In [6]:
nlp = spacy.load('en_core_web_sm', disable=['parser','ner'])

In [7]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word and perform lemmatization
        tokenized_words = nlp(pattern)
        words.extend(tokenized_words)
        # Add documents to the corpus
        documents.append((tokenized_words, intent['tag']))
        # Add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [9]:
print('Tokenize words -- ',tokenized_words)
print('words -- ',words)
print('documents -- ',documents)
print('\n')
print( 'length',len(tokenized_words))
print( 'length',len(words))
print( 'length',len(documents))


Tokenize words --  How can I get in touch with codeclause ?
words --  [Greetings, Hey, ,, chatbot, Hi, Hi, there, How, are, you, Is, anyone, there, ?, Hey, Hola, Hello, Good, day, Bye, See, you, later, Goodbye, Nice, chatting, to, you, ,, bye, Till, next, time, Catch, you, later, See, you, soon, Goodnight, Thanks, Thank, you, That, 's, helpful, Awesome, ,, thanks, Thanks, for, helping, me, Thank, you, very, much, I, appreciate, it, Thanks, for, your, time, /, help, You, 're, great, ,, thanks, !, How, you, could, help, me, ?, What, you, can, do, ?, What, help, you, provide, ?, How, you, can, be, helpful, ?, What, support, is, offered, Where, to, apply, for, internship, ?, Internships, offered, How, to, apply, for, an, internship, ?, How, can, I, apply, for, the, internship, ?, How, can, I, get, more, informations, about, the, internships, ?, How, long, the, internship, can, be, ?, What, is, the, duration, of, the, internship, ?, How, many, fields, are, there, ?, Which, field, that, the,

In [24]:
# for intent in intents['intents']:
#     for pattern in intent['patterns']:

#         #tokenize each word
#         w = nltk.word_tokenize(pattern)
#         words.extend(w)
#         #add documents in the corpus
#         documents.append((w, intent['tag']))

#         # add to our classes list
#         if intent['tag'] not in classes:
#             classes.append(intent['tag'])

In [25]:
# print('Tokenize words -- ',tokenized_words)
# print('words -- ',words)
# print('documents -- ',documents)
# print('\n')
# print( 'length',len(tokenized_words))
# print( 'length',len(words))
# print( 'length',len(documents))

Tokenize words --  How can I get in touch with codeclause ?
words --  [Greetings, Hey, ,, chatbot, Hi, Hi, there, How, are, you, Is, anyone, there, ?, Hey, Hola, Hello, Good, day, Bye, See, you, later, Goodbye, Nice, chatting, to, you, ,, bye, Till, next, time, Catch, you, later, See, you, soon, Goodnight, Thanks, Thank, you, That, 's, helpful, Awesome, ,, thanks, Thanks, for, helping, me, Thank, you, very, much, I, appreciate, it, Thanks, for, your, time, /, help, You, 're, great, ,, thanks, !, How, you, could, help, me, ?, What, you, can, do, ?, What, help, you, provide, ?, How, you, can, be, helpful, ?, What, support, is, offered, Where, to, apply, for, internship, ?, Internships, offered, How, to, apply, for, an, internship, ?, How, can, I, apply, for, the, internship, ?, How, can, I, get, more, informations, about, the, internships, ?, How, long, the, internship, can, be, ?, What, is, the, duration, of, the, internship, ?, How, many, fields, are, there, ?, Which, field, that, the,

In [32]:
lemmatized_words = [token.lemma_ for token in nlp(" ".join(map(str, words)))if token.text not in ignore_words]
# Update the 'words' list with the lemmatized forms
# lemmatized_words = [token.lemma_ for token in words]
words = lemmatized_words
print(len(words))
print(lemmatized_words)

1410
['greet', 'hey', ',', 'chatbot', 'hi', 'hi', 'there', 'how', 'be', 'you', 'be', 'anyone', 'there', 'hey', 'Hola', 'hello', 'good', 'day', 'bye', 'see', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'till', 'next', 'time', 'catch', 'you', 'later', 'see', 'you', 'soon', 'Goodnight', 'thank', 'thank', 'you', 'that', 'be', 'helpful', 'Awesome', ',', 'thank', 'thank', 'for', 'help', 'I', 'thank', 'you', 'very', 'much', 'I', 'appreciate', 'it', 'thank', 'for', 'your', 'time', '/', 'help', 'you', 'be', 'great', ',', 'thank', 'how', 'you', 'could', 'help', 'I', 'what', 'you', 'can', 'do', 'what', 'help', 'you', 'provide', 'how', 'you', 'can', 'be', 'helpful', 'what', 'support', 'be', 'offer', 'where', 'to', 'apply', 'for', 'internship', 'internship', 'offer', 'how', 'to', 'apply', 'for', 'an', 'internship', 'how', 'can', 'I', 'apply', 'for', 'the', 'internship', 'how', 'can', 'I', 'get', 'more', 'information', 'about', 'the', 'internship', 'how', 'long', 'the', '

In [34]:
print(classes)
classes = sorted(list(set(classes)))
print(classes)

['greeting', 'goodbye', 'thanks', 'options', 'internships', 'internships duration', 'internships fields', 'About CodeClause', 'offer_letter', 'Contact']
['About CodeClause', 'Contact', 'goodbye', 'greeting', 'internships', 'internships duration', 'internships fields', 'offer_letter', 'options', 'thanks']


In [38]:
# document contain combination of pattern and intents
print(documents)
print (len(documents), "documents")

[(Greetings, 'greeting'), (Hey,chatbot, 'greeting'), (Hi, 'greeting'), (Hi there, 'greeting'), (How are you, 'greeting'), (Is anyone there?, 'greeting'), (Hey, 'greeting'), (Hola, 'greeting'), (Hello, 'greeting'), (Good day, 'greeting'), (Bye, 'goodbye'), (See you later, 'goodbye'), (Goodbye, 'goodbye'), (Nice chatting to you, bye, 'goodbye'), (Till next time, 'goodbye'), (Catch you later, 'goodbye'), (See you soon, 'goodbye'), (Goodnight, 'goodbye'), (Thanks, 'thanks'), (Thank you, 'thanks'), (That's helpful, 'thanks'), (Awesome, thanks, 'thanks'), (Thanks for helping me, 'thanks'), (Thank you very much, 'thanks'), (I appreciate it, 'thanks'), (Thanks for your time/help, 'thanks'), (You're great, thanks!, 'thanks'), (How you could help me?, 'options'), (What you can do?, 'options'), (What help you provide?, 'options'), (How you can be helpful?, 'options'), (What support is offered, 'options'), (Where to apply for internship?, 'internships'), (Internships offered, 'internships'), (How 

In [39]:
# classes contain intents
print(classes)
print (len(classes), "classes", classes)

['About CodeClause', 'Contact', 'goodbye', 'greeting', 'internships', 'internships duration', 'internships fields', 'offer_letter', 'options', 'thanks']
10 classes ['About CodeClause', 'Contact', 'goodbye', 'greeting', 'internships', 'internships duration', 'internships fields', 'offer_letter', 'options', 'thanks']


In [40]:
# words contain all tokenized words
print(words)
print (len(words), "lemmatized words", words)

['greet', 'hey', ',', 'chatbot', 'hi', 'hi', 'there', 'how', 'be', 'you', 'be', 'anyone', 'there', 'hey', 'Hola', 'hello', 'good', 'day', 'bye', 'see', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'till', 'next', 'time', 'catch', 'you', 'later', 'see', 'you', 'soon', 'Goodnight', 'thank', 'thank', 'you', 'that', 'be', 'helpful', 'Awesome', ',', 'thank', 'thank', 'for', 'help', 'I', 'thank', 'you', 'very', 'much', 'I', 'appreciate', 'it', 'thank', 'for', 'your', 'time', '/', 'help', 'you', 'be', 'great', ',', 'thank', 'how', 'you', 'could', 'help', 'I', 'what', 'you', 'can', 'do', 'what', 'help', 'you', 'provide', 'how', 'you', 'can', 'be', 'helpful', 'what', 'support', 'be', 'offer', 'where', 'to', 'apply', 'for', 'internship', 'internship', 'offer', 'how', 'to', 'apply', 'for', 'an', 'internship', 'how', 'can', 'I', 'apply', 'for', 'the', 'internship', 'how', 'can', 'I', 'get', 'more', 'information', 'about', 'the', 'internship', 'how', 'long', 'the', 'inter

In [50]:
# for an unique lemmatized words
words = list(set(words))

In [51]:
print(words)
print (len(words), "lemmatized words", words)

['helpful', 'you', 'what', 'chatbot', 'in', 'I', 'field', 'will', 'duration', 'Nice', 'next', 'get', 'domain', 'great', 'with', 'time', 'receieve', 'my', 'Goodbye', 'codeclause', 'be', 'support', 'more', 'bye', 'help', 'appreciate', 'offer', 'letter', 'that', 'service', 'it', 'of', 'many', 'concern', 'greet', 'good', 'Goodnight', 'to', 'do', 'anyone', 'see', 'thank', 'apply', ',', 'soon', 'tell', 'how', 'which', 'hey', 'internship', 'contact', 'much', 'hello', 'where', 'an', 'your', 'day', 'chatting', 'CodeClause', 'hi', 'there', 'till', 'the', 'have', 'not', 'long', 'Hola', 'can', 'later', 'could', 'for', 'very', 'catch', 'about', 'Awesome', 'information', 'provide', '/', 'touch', 'inernship', 'recieve', 'when']
82 lemmatized words ['helpful', 'you', 'what', 'chatbot', 'in', 'I', 'field', 'will', 'duration', 'Nice', 'next', 'get', 'domain', 'great', 'with', 'time', 'receieve', 'my', 'Goodbye', 'codeclause', 'be', 'support', 'more', 'bye', 'help', 'appreciate', 'offer', 'letter', 'that

In [53]:
pickle.dump(words,open('models/words.pkl','wb'))
pickle.dump(classes,open('models/classes.pkl','wb'))

------------------------------------------------------

In [54]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [87]:
for doc in documents:
    bag = []
    pattern_words = doc[0]
    # print('pattern + intent -> ',doc)
    # print('pattern ->',pattern_words)
    pattern_words = [token.lemma_ for token in nlp(" ".join(map(str, pattern_words)))if token.text not in ignore_words]
    # print('lemmaztized word ->',pattern_words)
    for w in words:
        # print('w',w)
        bag.append(1) if w in pattern_words else bag.append(0)
        # print('bag', bag)
        
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [88]:
training

[[[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]],
 [[0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0

In [93]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
training

C:\Users\Lakshya\AppData\Local\Temp\ipykernel_11320\303234037.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


array([[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])],
       [list([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])],
       [list([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
       ...,
       [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [95]:
x_train = list(training[:,0])
y_train = list(training[:,1])


In [100]:
print(x_train)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 

In [99]:
print(y_train)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 

--------------------------------------------------

In [101]:
model = Sequential()
model.add(Dense(128, input_shape=(len(x_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

In [103]:
# Compile model.
sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [105]:
#fitting and saving the model 
history = model.fit(np.array(x_train), np.array(y_train), epochs=200, batch_size=5, verbose=1)
model.save('models/nlu/chatbot_model.h5', history)

Epoch 1/200
149/149 [==============================] - 0s 3ms/step - loss: 0.0663 - accuracy: 0.9758
Epoch 2/200
149/149 [==============================] - 1s 4ms/step - loss: 0.0507 - accuracy: 0.9785
Epoch 3/200
149/149 [==============================] - 0s 3ms/step - loss: 0.0632 - accuracy: 0.9704
Epoch 4/200
149/149 [==============================] - 0s 2ms/step - loss: 0.0531 - accuracy: 0.9758
Epoch 5/200
149/149 [==============================] - 0s 3ms/step - loss: 0.0487 - accuracy: 0.9758
Epoch 6/200
149/149 [==============================] - 0s 2ms/step - loss: 0.0565 - accuracy: 0.9758
Epoch 7/200
149/149 [==============================] - 0s 2ms/step - loss: 0.0551 - accuracy: 0.9731
Epoch 8/200
149/149 [==============================] - 0s 3ms/step - loss: 0.0498 - accuracy: 0.9812
Epoch 9/200
149/149 [==============================] - 0s 3ms/step - loss: 0.0519 - accuracy: 0.9772
Epoch 10/200
149/149 [==============================] - 0s 3ms/step - loss: 0.0422 - accura

-------------------------------------------------------------